# Pull Odds

In [34]:
import os
import requests
import json

API_KEY = os.getenv("ODDS_API_KEY", "3b1982ba40267fc53278ccf198874743")

url = "https://api.the-odds-api.com/v4/sports/upcoming/odds/"
params = {
    "apiKey": API_KEY,
    "sport": "tennis_atp_french_open",
    "regions": "us",          # us | uk | eu | au | us2
    "markets": "h2h",         # h2h, spreads, totals, outrights, etc.
    "oddsFormat": "american",    # decimal | american
    "dateFormat": "iso",         # iso | unix
}

resp = requests.get(url, params=params)
resp.raise_for_status()
games = resp.json()

In [35]:
with open("french_open_odds_outright.json", "w") as f:
    json.dump(games, f, indent=2)

# Positive EV Bets

In [36]:
import csv
import json
import math

# ———————— 1) load model predictions ——————————————
# We assume proba = P(player1 wins); so P(player2 wins) = 1 - proba
preds = {}
with open("roland_predictions.csv", newline="") as f:
    reader = csv.DictReader(f)
    for row in reader:
        p1_name = row["player1_name"]
        p2_name = row["player2_name"]
        p1_prob = float(row["proba"])
        p2_prob = 1.0 - p1_prob
        # store both (player1,player2) and reversed keys for easy lookup
        preds[(p1_name, p2_name)] = (p1_prob, p2_prob)
        preds[(p2_name, p1_name)] = (p2_prob, p1_prob)

In [37]:
# ———————— 2) load odds JSON ————————————————————————
with open("french_open_odds_pretty.json") as f:
    events = json.load(f)

In [38]:
# ———————— 3) helper: implied probability from American odds ——
def implied_prob_from_american(odds: float) -> float:
    """
    Convert American odds to implied probability.
        - Positive odds:  +150 → 100 / (150 + 100) = 0.40
        - Negative odds:  -200 → 200 / (200 + 100) = 0.66…
    """
    odds = float(odds)
    if odds > 0:
        return 100.0 / (odds + 100.0)
    else:
        return -odds / (-odds + 100.0)

In [39]:
# ———————— 4) scan for positive EV bets —————————————
ev_bets = []

for ev in events:
    home = ev["home_team"]
    away = ev["away_team"]

    # skip if we have no model prediction for this pairing
    if (home, away) not in preds:
        continue

    p_home, p_away = preds[(home, away)]

    for book in ev.get("bookmakers", []):
        book_title = book["title"]
        for market in book.get("markets", []):
            if market["key"] != "h2h":
                continue
            for outcome in market.get("outcomes", []):
                name = outcome["name"]
                odds = outcome["price"]
                imp = implied_prob_from_american(odds)
                model_p = p_home if name == home else p_away

                ev_pct = model_p / imp - 1.0
                if ev_pct > 0:
                    ev_bets.append({
                        "match": f"{home} vs {away}",
                        "bookmaker": book_title,
                        "selection": name,
                        "odds": odds,
                        "model_prob": model_p,
                        "implied_prob": imp,
                        "ev": ev_pct,
                    })

In [40]:
# ———————— 5) pretty‑print the positive EV bets ————————
# sort by highest EV first
ev_bets.sort(key=lambda x: x["ev"], reverse=True)

print(f"\nPositive-EV bets found: {len(ev_bets)}\n")
print(f"{'Match':30s} {'Bookmaker':15s} {'Pick':20s} {'Odds':6s} {'EV%':>6s}")
print("-" * 85)
for b in ev_bets:
    ev_display = b["ev"] * 100
    print(f"{b['match']:30s} {b['bookmaker']:15s} {b['selection']:20s} "
            f"{b['odds']:6d} {ev_display:6.1f}%")


Positive-EV bets found: 0

Match                          Bookmaker       Pick                 Odds      EV%
-------------------------------------------------------------------------------------
